**Programming for Big Data Project by Ben Fitzgerald (s00244687)**

In [1]:
#install pyspark

!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=1f05d4bc30a903cb964a3d2f20e19cceb6e8ae77c3ab83fe820677ba5be22adf
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

#create a SparkSession
spark = SparkSession.builder \
    .appName("NFL Combine Data Analysis") \
    .getOrCreate()


Import in NFL Combine and NFL Draft datasets

In [3]:
from google.colab import drive

#mount google drive
drive.mount('/content/drive')

# path to both draft and combine files stored on my google drive
combine_file_path = '/content/drive/My Drive/Big Data/combine_data.csv'
draft_file_path = '/content/drive/My Drive/Big Data/draft_data.csv'

#read CSV file into Spark DataFrame
combine_data = spark.read.format("csv").option("header", "true").load(combine_file_path)
draft_data = spark.read.format("csv").option("header", "true").load(draft_file_path)




Mounted at /content/drive


In [4]:
#Show first few rows of the combine dataframe
combine_data.show()


+----------------+---+-----------------+-------------+------+------+-----+--------+-----+-----+---------+-------+--------------------+--------------------+--------------------+----+
|          player|pos|          college|        stats|height|weight|forty|vertical|bench|broad|threecone|shuttle|             drafted|   college_stats_url|           nfl_stats|Year|
+----------------+---+-----------------+-------------+------+------+-----+--------+-----+-----+---------+-------+--------------------+--------------------+--------------------+----+
|    John Abraham|OLB|   South Carolina|         NULL|   6-4|   252| 4.55|    NULL| NULL| NULL|     NULL|   NULL|New York Jets / 1...|                NULL|https://www.pro-f...|2000|
| Shaun Alexander| RB|          Alabama|College Stats|   6-0|   218| 4.58|    NULL| NULL| NULL|     NULL|   NULL|Seattle Seahawks ...|https://www.sport...|https://www.pro-f...|2000|
|  Darnell Alford| OT|      Boston Col.|         NULL|   6-4|   334| 5.56|    25.0|   23| 

In [5]:
#Show first few rows of the draft dataframe

draft_data.show()

+-----+----+----+--------------------+---+---+----+---+---+---+-----+----+-----+--------+--------+--------+--------+---------+--------+--------+--------+----------+-------+-------+-------+----+-----+--------------+-------------+--------------------+--------------------+----+
|round|pick|team|              player|pos|age|  to|ap1| pb| st|carav|drav|games|pass_cmp|pass_att|pass_yds|pass_tds|pass_ints|rush_att|rush_yds|rush_tds|receptions|rec_yds|rec_tds|tackles|ints|sacks|       college|        stats|                 url|          player_url|Year|
+-----+----+----+--------------------+---+---+----+---+---+---+-----+----+-----+--------+--------+--------+--------+---------+--------+--------+--------+----------+-------+-------+-------+----+-----+--------------+-------------+--------------------+--------------------+----+
|    1|   1| CLE|      Courtney Brown| DE| 22|2005|  0|  0|  4|   27|  21|   61|       0|       0|       0|       0|        0|       0|       0|       0|         0|      0|

In [6]:
#drop the stats column from both which was only used to extract the college stats URLs
combine_data = combine_data.drop("stats")
draft_data = draft_data.drop("stats")

In [7]:
from pyspark.sql.functions import col

# Rename common columns in combine and draft dataframe to differentiate the two
combine_data = combine_data.withColumnRenamed("player", "combine_player") \
                           .withColumnRenamed("pos", "combine_pos") \
                           .withColumnRenamed("college", "combine_college") \
                           .withColumnRenamed("year", "combine_year")


draft_data = draft_data.withColumnRenamed("player", "draft_player") \
                       .withColumnRenamed("pos", "draft_pos") \
                       .withColumnRenamed("college", "draft_college") \
                       .withColumnRenamed("year", "draft_year")


In [8]:
from pyspark.sql.functions import col, regexp_replace

# Replace "http" with "https" in the 'url' and 'player_url' columns of the draft data frame for some reason it scraped the url with http
draft_data = draft_data.withColumn("url", regexp_replace(col("url"), "http://", "https://")) \
                       .withColumn("player_url", regexp_replace(col("player_url"), "http://", "https://"))


pattern_to_replace = "https://www.pro-football-reference.com//players/"

# Replace the pattern in the "nfl_stats" column which holds the players nfl stats
combine_data = combine_data.withColumn("nfl_stats", regexp_replace("nfl_stats", pattern_to_replace, "https://www.pro-football-reference.com/players/"))

In [9]:
# Perform the outer merge operation based on matching nfl stats or college stats url
merged_data = combine_data.join(draft_data, (combine_data["college_stats_url"] == draft_data["url"]) |
                                (combine_data["nfl_stats"] == draft_data["player_url"]), "outer")

# Show the resulting DataFrame
merged_data.show()


+----------------+-----------+-----------------+------+------+-----+--------+-----+-----+---------+-------+--------------------+--------------------+--------------------+------------+-----+----+----+----------------+---------+----+----+----+----+----+-----+----+-----+--------+--------+--------+--------+---------+--------+--------+--------+----------+-------+-------+-------+----+-----+---------------+--------------------+--------------------+----------+
|  combine_player|combine_pos|  combine_college|height|weight|forty|vertical|bench|broad|threecone|shuttle|             drafted|   college_stats_url|           nfl_stats|combine_year|round|pick|team|    draft_player|draft_pos| age|  to| ap1|  pb|  st|carav|drav|games|pass_cmp|pass_att|pass_yds|pass_tds|pass_ints|rush_att|rush_yds|rush_tds|receptions|rec_yds|rec_tds|tackles|ints|sacks|  draft_college|                 url|          player_url|draft_year|
+----------------+-----------+-----------------+------+------+-----+--------+-----+---

In [10]:
from pyspark.sql.functions import coalesce

# Create new columns by combining corresponding columns
merged_data = merged_data.withColumn("player", coalesce(merged_data["draft_player"], merged_data["combine_player"])) \
                         .withColumn("pos", coalesce(merged_data["draft_pos"], merged_data["combine_pos"])) \
                         .withColumn("college", coalesce(merged_data["draft_college"], merged_data["combine_college"])) \
                         .withColumn("year", coalesce(merged_data["draft_year"], merged_data["combine_year"]))


In [11]:
# Drop the original 8 columns
merged_data = merged_data.drop("draft_player", "draft_pos", "draft_college", "draft_year",
                               "combine_player", "combine_pos", "combine_college", "combine_year")


In [12]:
from pyspark.sql.functions import split, col

# Splitting the 'height' column into 'feet' and 'inches'
height_split = split(col("height"), "-")
merged_data = merged_data.withColumn("feet", height_split[0].cast("int"))
merged_data = merged_data.withColumn("inches", height_split[1].cast("int"))

# Converting feet to inches and adding to inches
merged_data = merged_data.withColumn("total_height_inches", merged_data["feet"] * 12 + merged_data["inches"])

#drop the 'feet' and 'inches' columns if needed
merged_data = merged_data.drop("feet", "inches", "height")

merged_data.show()


+------+-----+--------+-----+-----+---------+-------+--------------------+--------------------+--------------------+-----+----+----+----+----+----+----+----+-----+----+-----+--------+--------+--------+--------+---------+--------+--------+--------+----------+-------+-------+-------+----+-----+--------------------+--------------------+----------------+---+---------------+----+-------------------+
|weight|forty|vertical|bench|broad|threecone|shuttle|             drafted|   college_stats_url|           nfl_stats|round|pick|team| age|  to| ap1|  pb|  st|carav|drav|games|pass_cmp|pass_att|pass_yds|pass_tds|pass_ints|rush_att|rush_yds|rush_tds|receptions|rec_yds|rec_tds|tackles|ints|sacks|                 url|          player_url|          player|pos|        college|year|total_height_inches|
+------+-----+--------+-----+-----+---------+-------+--------------------+--------------------+--------------------+-----+----+----+----+----+----+----+----+-----+----+-----+--------+--------+--------+---

In [13]:
from pyspark.sql.functions import when, col

# Create a new column "Drafted" based on the condition
merged_data = merged_data.withColumn("Drafted", when(col("pick").isNull(), 0).otherwise(1))


In [14]:
# Assuming merged_data is your PySpark DataFrame
merged_data.printSchema()


root
 |-- weight: string (nullable = true)
 |-- forty: string (nullable = true)
 |-- vertical: string (nullable = true)
 |-- bench: string (nullable = true)
 |-- broad: string (nullable = true)
 |-- threecone: string (nullable = true)
 |-- shuttle: string (nullable = true)
 |-- Drafted: integer (nullable = false)
 |-- college_stats_url: string (nullable = true)
 |-- nfl_stats: string (nullable = true)
 |-- round: string (nullable = true)
 |-- pick: string (nullable = true)
 |-- team: string (nullable = true)
 |-- age: string (nullable = true)
 |-- to: string (nullable = true)
 |-- ap1: string (nullable = true)
 |-- pb: string (nullable = true)
 |-- st: string (nullable = true)
 |-- carav: string (nullable = true)
 |-- drav: string (nullable = true)
 |-- games: string (nullable = true)
 |-- pass_cmp: string (nullable = true)
 |-- pass_att: string (nullable = true)
 |-- pass_yds: string (nullable = true)
 |-- pass_tds: string (nullable = true)
 |-- pass_ints: string (nullable = true)
 |-

In [15]:
from pyspark.sql.functions import col

# List of columns to convert to float
columns_to_convert_to_float = [
 "forty", "vertical", "threecone", "shuttle", "sacks"
]

# List of columns to convert to integer
columns_to_convert_to_int = [
    "weight", "bench", "broad", "round", "pick", "age", "to", "ap1", "pb", "st",
    "carav", "drav", "games", "pass_cmp", "pass_att", "pass_yds",
    "pass_tds", "pass_ints", "rush_att", "rush_yds", "rush_tds",
    "receptions", "rec_yds", "rec_tds", "tackles", "ints", "year"
]

# Convert columns to float
for column in columns_to_convert_to_float:
    merged_data = merged_data.withColumn(column, col(column).cast("float"))

# Convert columns to integer
for column in columns_to_convert_to_int:
    merged_data = merged_data.withColumn(column, col(column).cast("int"))


Importing the College Stats from all Running Backs in the dataframe

In [16]:
#path to where I have stored the scraped running back data
rb_file_path = '/content/drive/My Drive/Big Data/rb_data.csv'

# Read CSV file into Spark DataFrame
rb_data = spark.read.format("csv").option("header", "true").load(rb_file_path)


In [17]:
rb_data.show()

+--------------------+----+----------------+--------+-----+----+----+----------------+----------------+------------------+---------------+---------------+---------------+------------------+--------------+-------------------+-----------------+-----------------+----------------+
|                 URL|Year|          School|    Conf|Class| Pos|   G|college_rush_att|college_rush_yds|  college_rush_avg|college_rush_TD|college_rec_rec|college_rec_yds|   college_rec_avg|college_rec_TD|college_scrim_plays|college_scrim_yds|college_scrim_avg|college_scrim_TD|
+--------------------+----+----------------+--------+-----+----+----+----------------+----------------+------------------+---------------+---------------+---------------+------------------+--------------+-------------------+-----------------+-----------------+----------------+
|https://www.sport...|NULL|             TCU|     MWC| NULL|NULL|42.0|           487.0|          2596.0|               5.3|           20.0|           81.0|          85

In [18]:
# Drop the columns "school", "class", and "pos"
rb_data = rb_data.drop("School", "Class", "Pos", "Year")


In [19]:
rb_data.printSchema()

root
 |-- URL: string (nullable = true)
 |-- Conf: string (nullable = true)
 |-- G: string (nullable = true)
 |-- college_rush_att: string (nullable = true)
 |-- college_rush_yds: string (nullable = true)
 |-- college_rush_avg: string (nullable = true)
 |-- college_rush_TD: string (nullable = true)
 |-- college_rec_rec: string (nullable = true)
 |-- college_rec_yds: string (nullable = true)
 |-- college_rec_avg: string (nullable = true)
 |-- college_rec_TD: string (nullable = true)
 |-- college_scrim_plays: string (nullable = true)
 |-- college_scrim_yds: string (nullable = true)
 |-- college_scrim_avg: string (nullable = true)
 |-- college_scrim_TD: string (nullable = true)



In [20]:
from pyspark.sql.functions import col

# List of columns to convert to float
columns_to_convert_to_float = [
    "college_rush_avg", "college_rec_avg", "college_scrim_avg"
]

# List of columns to convert to integer
columns_to_convert_to_int = [
    "G", "college_rush_att", "college_rush_yds", "college_rush_TD",
    "college_rec_rec", "college_rec_yds", "college_rec_TD",
    "college_scrim_plays", "college_scrim_yds", "college_scrim_TD"
]

# Convert columns to float
for column in columns_to_convert_to_float:
    rb_data = rb_data.withColumn(column, col(column).cast("float"))

# Convert columns to integer
for column in columns_to_convert_to_int:
    rb_data = rb_data.withColumn(column, col(column).cast("int"))



In [21]:
# Rename the column "URL" to "college_url"
rb_data = rb_data.withColumnRenamed("URL", "college_url")


In [22]:
#from the nfl combine and nfl draft combined dataset we only want running backs

just_rb_merged = merged_data.filter(merged_data["pos"] == "RB")
just_rb_merged.show()

+------+-----+--------+-----+-----+---------+-------+-------+--------------------+--------------------+-----+----+----+----+----+----+----+----+-----+----+-----+--------+--------+--------+--------+---------+--------+--------+--------+----------+-------+-------+-------+----+-----+--------------------+--------------------+---------------+---+--------------------+----+-------------------+
|weight|forty|vertical|bench|broad|threecone|shuttle|Drafted|   college_stats_url|           nfl_stats|round|pick|team| age|  to| ap1|  pb|  st|carav|drav|games|pass_cmp|pass_att|pass_yds|pass_tds|pass_ints|rush_att|rush_yds|rush_tds|receptions|rec_yds|rec_tds|tackles|ints|sacks|                 url|          player_url|         player|pos|             college|year|total_height_inches|
+------+-----+--------+-----+-----+---------+-------+-------+--------------------+--------------------+-----+----+----+----+----+----+----+----+-----+----+-----+--------+--------+--------+--------+---------+--------+------

Combine the merged NFL Draft and NFL Combine dataset which has been filtered to only include Running Backs with the College Stats for Running Backs

In [23]:
all_rb_df = just_rb_merged.join(rb_data,
                                   just_rb_merged["college_stats_url"] == rb_data["college_url"],
                                   "inner")

# Show the resulting combined DataFrame
all_rb_df.show()


+------+-----+--------+-----+-----+---------+-------+-------+--------------------+--------------------+-----+----+----+----+----+----+----+----+-----+----+-----+--------+--------+--------+--------+---------+--------+--------+--------+----------+-------+-------+-------+----+-----+--------------------+--------------------+---------------+---+----------------+----+-------------------+--------------------+--------+---+----------------+----------------+----------------+---------------+---------------+---------------+---------------+--------------+-------------------+-----------------+-----------------+----------------+
|weight|forty|vertical|bench|broad|threecone|shuttle|Drafted|   college_stats_url|           nfl_stats|round|pick|team| age|  to| ap1|  pb|  st|carav|drav|games|pass_cmp|pass_att|pass_yds|pass_tds|pass_ints|rush_att|rush_yds|rush_tds|receptions|rec_yds|rec_tds|tackles|ints|sacks|                 url|          player_url|         player|pos|         college|year|total_height_i

In [24]:
 # get a count for how many players are now in my dataframe with college statistics
total_rows = all_rb_df.count()

print("Total number of rows in all_rb_df:", total_rows)


Total number of rows in all_rb_df: 633


In [25]:
from pyspark.sql.functions import col

# Check for null or blank values in college statistics
blank_columns = ["college_rush_att", "college_rush_yds", "college_rush_avg", "college_rush_TD",
                 "college_rec_rec", "college_rec_yds", "college_rec_avg", "college_rec_TD",
                 "college_scrim_plays", "college_scrim_yds", "college_scrim_avg", "college_scrim_TD"]

for column in blank_columns:
    blank_count = all_rb_df.filter((col(column).isNull()) | (col(column) == "")).count()
    print(f"Blank values in '{column}': {blank_count}")


Blank values in 'college_rush_att': 0
Blank values in 'college_rush_yds': 0
Blank values in 'college_rush_avg': 0
Blank values in 'college_rush_TD': 0
Blank values in 'college_rec_rec': 0
Blank values in 'college_rec_yds': 0
Blank values in 'college_rec_avg': 2
Blank values in 'college_rec_TD': 0
Blank values in 'college_scrim_plays': 0
Blank values in 'college_scrim_yds': 0
Blank values in 'college_scrim_avg': 0
Blank values in 'college_scrim_TD': 0


In [26]:
#if a player has NULL in college_rec_avg it means 0 so this is easy to fix

all_rb_df = all_rb_df.fillna({'college_rec_avg': 0})

In [27]:
from pyspark.sql.functions import col

# Check for null or blank values in combine statistics
blank_columns = ["total_height_inches", "weight", "forty", "vertical", "bench", "broad", "threecone", "shuttle"]

for column in blank_columns:
    blank_count = all_rb_df.filter((col(column).isNull()) | (col(column) == "")).count()
    print(f"Blank values in '{column}': {blank_count}")


Blank values in 'total_height_inches': 2
Blank values in 'weight': 1
Blank values in 'forty': 33
Blank values in 'vertical': 108
Blank values in 'bench': 186
Blank values in 'broad': 123
Blank values in 'threecone': 297
Blank values in 'shuttle': 290


In an ideal world - all college statistics would be there but not every player does every drill at the combine especially drills that aren't relevant to a specific position like running back. Height and Inches were missing for some players during covid years too where not all measurements were taken.

I found a website online that includes information for players from their Pro Days which were popular during Covid so included that information here.

If you don't complete your workout at the NFL Combine due to field conditions, injuries or other factors, you can showcase your skills at a Pro Day. Pro Days take place on university campuses and are more personal.

In [28]:
extra_rb_combine_file_path = '/content/drive/My Drive/Big Data/extra_combine_rb.csv'


# read CSV file into Spark DataFrame
extra_rb_combine_data = spark.read.format("csv").option("header", "true").load(extra_rb_combine_file_path)

In [29]:
extra_rb_combine_data.show()

# Print the schema
extra_rb_combine_data.printSchema()

+-------+---------------+------+-----------+----+-----------+----+----------+-----------+-----+
|Year_RB|           Name|Height|Weight (in)|  40|Bench Press|Vert|Broad Jump|Shuttle Run|3Cone|
+-------+---------------+------+-----------+----+-----------+----+----------+-----------+-----+
|   2024|    Rasheen Ali| 71.38|        206|NULL|       NULL|NULL|      NULL|       NULL| NULL|
|   2024|  Braelon Allen|    73|        235|NULL|         26|  32|       117|       NULL| NULL|
|   2024|   Emani Bailey| 67.38|        202|4.61|       NULL|33.5|       116|       NULL| NULL|
|   2024|    Trey Benson| 72.38|        216|4.39|       NULL|33.5|       122|       NULL| NULL|
|   2024|Jonathon Brooks|    72|        216|NULL|       NULL|NULL|      NULL|       NULL| NULL|
|   2024|    Blake Corum| 67.63|        205|4.53|         27|35.5|      NULL|       NULL| NULL|
|   2024|   Isaiah Davis| 71.75|        218|4.57|         23|34.5|       119|       NULL| NULL|
|   2024|      Ray Davis| 68.38|        

In [30]:
from pyspark.sql.functions import col

# Convert columns to float
float_columns = ["Height", "40", "Vert", "Shuttle Run", "3Cone"]
for column in float_columns:
    extra_rb_combine_data = extra_rb_combine_data.withColumn(column, col(column).cast("float"))

# Convert columns to int
int_columns = ["Weight (in)", "Bench Press", "Broad Jump"]
for column in int_columns:
    extra_rb_combine_data = extra_rb_combine_data.withColumn(column, col(column).cast("int"))


In [31]:
from pyspark.sql.functions import coalesce

# Join the dataframes based on player name and year
joined_df = all_rb_df.join(extra_rb_combine_data,
                           (all_rb_df["player"] == extra_rb_combine_data["Name"]) &
                           (all_rb_df["year"] == extra_rb_combine_data["Year_RB"]),
                           "left_outer")

# Replace blank values in specific columns with corresponding values from other columns
columns_to_replace = [
    ("total_height_inches", "Height"),
    ("weight", "Weight (in)"),
    ("forty", "40"),
    ("vertical", "Vert"),
    ("bench", "Bench Press"),
    ("broad", "Broad Jump"),
    ("threecone", "3Cone"),
    ("shuttle", "Shuttle Run")
]

for target_column, source_column in columns_to_replace:
    joined_df = joined_df.withColumn(target_column, coalesce(joined_df[target_column], joined_df[source_column]))

# Drop the redundant columns
columns_to_drop = ["Name", "Year_RB", "Height", "Weight (in)", "40", "Vert", "Bench Press", "Broad Jump", "3Cone", "Shuttle Run"]
joined_df = joined_df.drop(*columns_to_drop)

# Show the updated dataframe
joined_df.show()


+------+-----+--------+-----+-----+---------+-------+-------+--------------------+--------------------+-----+----+----+----+----+----+----+----+-----+----+-----+--------+--------+--------+--------+---------+--------+--------+--------+----------+-------+-------+-------+----+-----+--------------------+--------------------+---------------+---+----------------+----+-------------------+--------------------+--------+---+----------------+----------------+----------------+---------------+---------------+---------------+---------------+--------------+-------------------+-----------------+-----------------+----------------+
|weight|forty|vertical|bench|broad|threecone|shuttle|Drafted|   college_stats_url|           nfl_stats|round|pick|team| age|  to| ap1|  pb|  st|carav|drav|games|pass_cmp|pass_att|pass_yds|pass_tds|pass_ints|rush_att|rush_yds|rush_tds|receptions|rec_yds|rec_tds|tackles|ints|sacks|                 url|          player_url|         player|pos|         college|year|total_height_i

Recheck Null values to see if the Pro Day stats added anything to the dataframe

In [32]:
from pyspark.sql.functions import col

# Check for null or blank values in specified columns
blank_columns = ["college_rush_att", "college_rush_yds", "college_rush_avg", "college_rush_TD",
                 "college_rec_rec", "college_rec_yds", "college_rec_avg", "college_rec_TD",
                 "college_scrim_plays", "college_scrim_yds", "college_scrim_avg", "college_scrim_TD"]

for column in blank_columns:
    blank_count = joined_df.filter((col(column).isNull()) | (col(column) == "")).count()
    print(f"Blank values in '{column}': {blank_count}")


Blank values in 'college_rush_att': 0
Blank values in 'college_rush_yds': 0
Blank values in 'college_rush_avg': 0
Blank values in 'college_rush_TD': 0
Blank values in 'college_rec_rec': 0
Blank values in 'college_rec_yds': 0
Blank values in 'college_rec_avg': 0
Blank values in 'college_rec_TD': 0
Blank values in 'college_scrim_plays': 0
Blank values in 'college_scrim_yds': 0
Blank values in 'college_scrim_avg': 0
Blank values in 'college_scrim_TD': 0


In [33]:
from pyspark.sql.functions import col

# Check for null or blank values in specified columns
blank_columns = ["total_height_inches", "weight", "forty", "vertical", "bench", "broad", "threecone", "shuttle"]

for column in blank_columns:
    blank_count = joined_df.filter((col(column).isNull()) | (col(column) == "")).count()
    print(f"Blank values in '{column}': {blank_count}")


Blank values in 'total_height_inches': 0
Blank values in 'weight': 0
Blank values in 'forty': 23
Blank values in 'vertical': 69
Blank values in 'bench': 135
Blank values in 'broad': 90
Blank values in 'threecone': 209
Blank values in 'shuttle': 197


In [34]:
all_rb_df = joined_df

In [35]:
# Drop rows with NULL 40 yard dash time - running backs who didn' complete a 40 yard dash time are normally injured as this is the most important metric for a running back
all_rb_df = all_rb_df.filter(col("40").isNotNull())




In [36]:
from pyspark.sql.functions import col

# Count the occurrences of "Yes" and "No" in the "Drafted" column and print the output
drafted_yes_count = all_rb_df.filter(col("Drafted") == 1).count()
drafted_no_count = all_rb_df.filter(col("Drafted") == 0).count()

print("Count of 'Yes' in Drafted column:", drafted_yes_count)
print("Count of 'No' in Drafted column:", drafted_no_count)


Count of 'Yes' in Drafted column: 330
Count of 'No' in Drafted column: 200


In [37]:
# Get unique list of conference names that players played in in College and print the output
unique_conferences = all_rb_df.select("Conf").distinct().rdd.map(lambda row: row[0]).collect()

print("Unique Conference Names:")
for conf in unique_conferences:
    print(conf)



Unique Conference Names:
Big Ten
ACC
SEC
WAC
MWC
Sun Belt
CUSA
Pac-10
Pac-12
MAC
Big 12
Ind
Big West
Big East
American


Power 5 Conferences are seen as more pretigious, most players want to play at a Power 5 School

In [38]:


power_5_conferences = {"Big Ten", "ACC", "SEC", "Pac-12", "Big 12"}

# Convert "Conf" values to numerical values (1 for Power 5, 0 otherwise)
all_rb_df = all_rb_df.withColumn("Power_5_Num", when(col("Conf").isin(list(power_5_conferences)), 1).otherwise(0))


**Predicting Likelihood of a Running Back being drafted based on their Combine and College Statistics.**

In [39]:
from pyspark.sql.functions import corr

# Compute correlation between each feature column and the target variable
correlation_results = {}
for column in ["forty", "G", "college_rush_att","college_rush_yds","college_rush_avg","college_rush_TD",
               "college_rec_rec","college_rec_yds","college_rec_avg","college_rec_TD","college_scrim_plays",
               "college_scrim_avg", "college_scrim_yds", "college_scrim_TD", "Power_5_Num"]:
    correlation_value = all_rb_df.corr(column, "Drafted")
    correlation_results[column] = correlation_value

# Print correlation results
for column, correlation_value in correlation_results.items():
    print(f"Correlation between '{column}' and 'Drafted': {correlation_value}")

Correlation between 'forty' and 'Drafted': -0.3723759023902145
Correlation between 'G' and 'Drafted': 0.017209719175177697
Correlation between 'college_rush_att' and 'Drafted': 0.1491916215983347
Correlation between 'college_rush_yds' and 'Drafted': 0.22352916309313797
Correlation between 'college_rush_avg' and 'Drafted': 0.18928895413518415
Correlation between 'college_rush_TD' and 'Drafted': 0.23551586729898139
Correlation between 'college_rec_rec' and 'Drafted': 0.042566781433639504
Correlation between 'college_rec_yds' and 'Drafted': 0.09931685737270803
Correlation between 'college_rec_avg' and 'Drafted': 0.1240170596290748
Correlation between 'college_rec_TD' and 'Drafted': 0.10612749208639789
Correlation between 'college_scrim_plays' and 'Drafted': 0.14524138857984778
Correlation between 'college_scrim_avg' and 'Drafted': 0.1676667191373251
Correlation between 'college_scrim_yds' and 'Drafted': 0.2239090554353721
Correlation between 'college_scrim_TD' and 'Drafted': 0.24228546457

There seems to be a correlation between forty time and whether a player is drafted or not. The quicker the forty time, the more likely they are to be drafted which makes sense. A lot of the college statistics also look like they matter with the higher values representing a better chance of being drafted.

In [40]:
power_5_counts = all_rb_df.groupBy("Power_5_Num").count()

# Show the breakdown of 1s and 0s in the "Power_5_Num" column
power_5_counts.show()

+-----------+-----+
|Power_5_Num|count|
+-----------+-----+
|          1|  340|
|          0|  190|
+-----------+-----+



In [41]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Select relevant columns
selected_cols = ["forty", "college_rush_att", "college_rush_yds","college_rush_avg", "college_scrim_TD", "Drafted","year", "player"]
selected_df = all_rb_df.select(selected_cols)

# Drop rows with null values
selected_df = selected_df.dropna()

# Filter the data for training and validation
train_validation_data = selected_df.filter(selected_df["year"] != 2023)

# Filter the data for testing
test_data = selected_df.filter(selected_df["year"] == 2023)

# Prepare features
assembler = VectorAssembler(inputCols=["forty", "college_rush_att", "college_rush_yds","college_rush_avg", "college_scrim_TD"],
                            outputCol="features")

# Prepare features for training and validation data
train_validation_assembled = assembler.transform(train_validation_data)

# Prepare features for test data
test_assembled = assembler.transform(test_data)

# Split the training and validation data
train_data, validation_data = train_validation_assembled.randomSplit([0.8, 0.2], seed=42)

# Train Logistic Regression model
lr = LogisticRegression(labelCol="Drafted", featuresCol="features")
model = lr.fit(train_data)

# Make predictions on validation data
validation_predictions = model.transform(validation_data)

# Evaluate the model on validation data
evaluator = BinaryClassificationEvaluator(labelCol="Drafted")
validation_auc = evaluator.evaluate(validation_predictions)
print("Validation Area Under ROC:", validation_auc)

# Make predictions on test data
test_predictions = model.transform(test_assembled)

# Evaluate the model on test data
test_auc = evaluator.evaluate(test_predictions)
print("Test Area Under ROC:", test_auc)

# Save the model
model_path = "/content/drive/My Drive/Big Data/logistic_regression_model_drafted_rb"
model.write().overwrite().save(model_path)




Validation Area Under ROC: 0.81981981981982
Test Area Under ROC: 0.6984126984126985


In [42]:
# Show predictions for 2023 NFL Draft data
test_predictions.select("player", "Drafted", "prediction", "probability").show(truncate=False)


+-------------------+-------+----------+----------------------------------------+
|player             |Drafted|prediction|probability                             |
+-------------------+-------+----------+----------------------------------------+
|Tank Bigsby        |1      |1.0       |[0.3523098686064784,0.6476901313935216] |
|Chase Brown        |1      |1.0       |[0.18054121483856483,0.8194587851614352]|
|Zach Charbonnet    |1      |1.0       |[0.16923699923207136,0.8307630007679286]|
|Travis Dye         |0      |0.0       |[0.5775218429570995,0.4224781570429005] |
|Tiyon Evans        |0      |1.0       |[0.4305149125035872,0.5694850874964128] |
|Jahmyr Gibbs       |1      |1.0       |[0.13071199691223195,0.8692880030877681]|
|Evan Hull          |1      |1.0       |[0.23498105376842762,0.7650189462315724]|
|Roschon Johnson    |1      |1.0       |[0.3884866062201133,0.6115133937798867] |
|Kenny McIntosh     |1      |0.0       |[0.5029034682676263,0.49709653173237367]|
|Keaton Mitchell

In [43]:
from pyspark.sql.functions import col

# Filter rows where Drafted is equal to 1
drafted_players = test_predictions

# Order by probability column in ascending order
drafted_players_ordered = drafted_players.orderBy(col("probability").asc())

# Create a new DataFrame with the selected columns and rename the player column
drafted_players_df = drafted_players_ordered.select(col("player").alias("player_name"), "probability")

# Show the result
drafted_players_df.show(truncate=False)




+-------------------+----------------------------------------+
|player_name        |probability                             |
+-------------------+----------------------------------------+
|Keaton Mitchell    |[0.07255755932954691,0.9274424406704531]|
|Bijan Robinson     |[0.07758199776383749,0.9224180022361625]|
|Jahmyr Gibbs       |[0.13071199691223195,0.8692880030877681]|
|Tyjae Spears       |[0.15687351153643916,0.8431264884635609]|
|Deneric Prince     |[0.1609460132548103,0.8390539867451897] |
|Zach Charbonnet    |[0.16923699923207136,0.8307630007679286]|
|Chase Brown        |[0.18054121483856483,0.8194587851614352]|
|Evan Hull          |[0.23498105376842762,0.7650189462315724]|
|Camerun Peoples    |[0.23651085845384537,0.7634891415461547]|
|Tank Bigsby        |[0.3523098686064784,0.6476901313935216] |
|Roschon Johnson    |[0.3884866062201133,0.6115133937798867] |
|Tiyon Evans        |[0.4305149125035872,0.5694850874964128] |
|Kenny McIntosh     |[0.5029034682676263,0.497096531732

In [44]:
# Join the drafted_players_df with all_rb_df based on the player's name
joined_df = drafted_players_df.join(all_rb_df, drafted_players_df["player_name"] == all_rb_df["player"], "inner")

# Select desired columns from the joined DataFrame
joined_df.select("player_name", "pick", "probability").orderBy(col("probability").asc()).show(truncate=False)





+-------------------+----+----------------------------------------+
|player_name        |pick|probability                             |
+-------------------+----+----------------------------------------+
|Keaton Mitchell    |NULL|[0.07255755932954691,0.9274424406704531]|
|Bijan Robinson     |8   |[0.07758199776383749,0.9224180022361625]|
|Jahmyr Gibbs       |12  |[0.13071199691223195,0.8692880030877681]|
|Tyjae Spears       |81  |[0.15687351153643916,0.8431264884635609]|
|Deneric Prince     |NULL|[0.1609460132548103,0.8390539867451897] |
|Zach Charbonnet    |52  |[0.16923699923207136,0.8307630007679286]|
|Chase Brown        |163 |[0.18054121483856483,0.8194587851614352]|
|Evan Hull          |176 |[0.23498105376842762,0.7650189462315724]|
|Camerun Peoples    |NULL|[0.23651085845384537,0.7634891415461547]|
|Tank Bigsby        |88  |[0.3523098686064784,0.6476901313935216] |
|Roschon Johnson    |115 |[0.3884866062201133,0.6115133937798867] |
|Tiyon Evans        |NULL|[0.4305149125035872,0.

**Predicting Career Success in the National Football League**

In [45]:
from pyspark.sql.functions import corr

# Filter the DataFrame for data where year < 2018 because some running backs drafted in the
#last 5 years might not have accumulated enough carav yet for the ability they have
filtered_data = all_rb_df.filter(all_rb_df["year"] < 2018)

# Compute correlation between each feature column and the target variable
correlation_results = {}
for column in ["forty", "G", "college_rush_att","college_rush_yds","college_rush_avg","college_rush_TD",
               "college_rec_rec","college_rec_yds","college_rec_avg","college_rec_TD","college_scrim_plays",
               "college_scrim_avg", "college_scrim_yds", "college_scrim_TD", "Power_5_Num"]:
    correlation_value = filtered_data.corr(column, "carav")
    correlation_results[column] = correlation_value

# Print correlation results
for column, correlation_value in correlation_results.items():
    print(f"Correlation between '{column}' and 'Carav': {correlation_value}")


Correlation between 'forty' and 'Carav': -0.2810088045937276
Correlation between 'G' and 'Carav': -0.09092055831244437
Correlation between 'college_rush_att' and 'Carav': 0.20411541299235864
Correlation between 'college_rush_yds' and 'Carav': 0.3027694671185301
Correlation between 'college_rush_avg' and 'Carav': 0.23798473497412645
Correlation between 'college_rush_TD' and 'Carav': 0.27780909677060966
Correlation between 'college_rec_rec' and 'Carav': 0.08513333422879102
Correlation between 'college_rec_yds' and 'Carav': 0.14325903312917135
Correlation between 'college_rec_avg' and 'Carav': 0.2050265364611634
Correlation between 'college_rec_TD' and 'Carav': 0.15559636882327996
Correlation between 'college_scrim_plays' and 'Carav': 0.2025138988405899
Correlation between 'college_scrim_avg' and 'Carav': 0.19949186698983468
Correlation between 'college_scrim_yds' and 'Carav': 0.3038323683981451
Correlation between 'college_scrim_TD' and 'Carav': 0.29110513067197047
Correlation between 'P

In [46]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# Filter training/validation and test sets
train_val_data = all_rb_df.filter((all_rb_df['year'] <= 2018) & (all_rb_df['carav'].isNotNull()) & (all_rb_df['Drafted'] == 1))
test_data = all_rb_df.filter(all_rb_df['year'] == 2023)

# Define the features and target variable
features = ['forty',"college_rush_att", "college_rush_avg", "college_scrim_TD", "college_scrim_yds"]
target = 'carav'

# Create a vector assembler to combine features into a single vector column
assembler = VectorAssembler(inputCols=features, outputCol="features")

# Transform the data using the vector assembler
train_val_data = assembler.transform(train_val_data)
test_data = assembler.transform(test_data)

# Define and train the linear regression model
lr = LinearRegression(featuresCol="features", labelCol=target)
lr_model = lr.fit(train_val_data)

# Make predictions on the test data
predictions = lr_model.transform(test_data)

# Select the player and the predicted carav columns
predicted_players = predictions.select('player', 'prediction', 'carav')

# Show the players with the predicted carav values
predicted_players.show()

# Define the path to save the model
model_path_carav = "/content/drive/My Drive/Big Data/linear_regression_model_carav_rb"

lr_model.write().overwrite().save(model_path_carav)


+-------------------+------------------+-----+
|             player|        prediction|carav|
+-------------------+------------------+-----+
|        Tank Bigsby| 16.66056993964426|    1|
|        Chase Brown| 22.26910728064884|    3|
|    Zach Charbonnet|23.703208358456635|    5|
|         Travis Dye|22.139310786694267| NULL|
|        Tiyon Evans| 7.102339029453873| NULL|
|       Jahmyr Gibbs|26.745040541257808|   10|
|          Evan Hull|20.607034790308887|    0|
|    Roschon Johnson|14.432042474460332|    4|
|     Kenny McIntosh|15.032549624515752|    0|
|    Keaton Mitchell| 27.94328753944366| NULL|
|    Camerun Peoples|17.272216034268638| NULL|
|     Deneric Prince|  16.4560733988843| NULL|
|     Bijan Robinson|30.104303523653826|    9|
|       Tyjae Spears| 24.31654558634017|    6|
|      Tavion Thomas| 6.867235730271787| NULL|
|SaRodorick Thompson|12.436798619366783| NULL|
+-------------------+------------------+-----+



In [47]:

# Order predicted players by highest prediction
predicted_players_ordered = predicted_players.orderBy(predicted_players.prediction.desc())

# Show the ordered predicted players
predicted_players_ordered.show()


+-------------------+------------------+-----+
|             player|        prediction|carav|
+-------------------+------------------+-----+
|     Bijan Robinson|30.104303523653826|    9|
|    Keaton Mitchell| 27.94328753944366| NULL|
|       Jahmyr Gibbs|26.745040541257808|   10|
|       Tyjae Spears| 24.31654558634017|    6|
|    Zach Charbonnet|23.703208358456635|    5|
|        Chase Brown| 22.26910728064884|    3|
|         Travis Dye|22.139310786694267| NULL|
|          Evan Hull|20.607034790308887|    0|
|    Camerun Peoples|17.272216034268638| NULL|
|        Tank Bigsby| 16.66056993964426|    1|
|     Deneric Prince|  16.4560733988843| NULL|
|     Kenny McIntosh|15.032549624515752|    0|
|    Roschon Johnson|14.432042474460332|    4|
|SaRodorick Thompson|12.436798619366783| NULL|
|        Tiyon Evans| 7.102339029453873| NULL|
|      Tavion Thomas| 6.867235730271787| NULL|
+-------------------+------------------+-----+



**Testing both models on 2024 Data**

In [48]:
# path to both draft and combine files stored on my google drive
combine_rb_path_2024 = '/content/drive/My Drive/Big Data/2024_combine_rb.csv'


# read CSV file into Spark DataFrame
combine_2024_rb = spark.read.format("csv").option("header", "true").load(combine_rb_path_2024)




In [49]:
combine_2024_rb.show()

+----------------+-----+--------+----+----------------+----------------+------------------+---------------+---------------+---------------+------------------+--------------+-------------------+-----------------+------------------+----------------+-----------+
|          player|forty|    Conf|   G|college_rush_att|college_rush_yds|  college_rush_avg|college_rush_TD|college_rec_rec|college_rec_yds|   college_rec_avg|college_rec_TD|college_scrim_plays|college_scrim_yds| college_scrim_avg|college_scrim_TD|Power_5_Num|
+----------------+-----+--------+----+----------------+----------------+------------------+---------------+---------------+---------------+------------------+--------------+-------------------+-----------------+------------------+----------------+-----------+
|    Emani Bailey| 4.61|  Big 12|42.0|           366.0|          2161.0| 6.449999999999999|           18.0|           47.0|          380.0|7.8999999999999995|           1.0|              413.0|           2541.0|         

In [50]:
from pyspark.sql.functions import col

# Define the list of columns to cast to float and int
float_columns = ["forty", "college_rush_avg"]
int_columns = ["college_rush_yds", "college_scrim_TD", "college_rush_att", "college_scrim_yds"]

# Cast float columns to float
for column in float_columns:
    combine_2024_rb = combine_2024_rb.withColumn(column, col(column).cast("float"))

# Cast int columns to integer
for column in int_columns:
    combine_2024_rb = combine_2024_rb.withColumn(column, col(column).cast("int"))

# Show the updated DataFrame schema
combine_2024_rb.printSchema()


root
 |-- player: string (nullable = true)
 |-- forty: float (nullable = true)
 |-- Conf: string (nullable = true)
 |-- G: string (nullable = true)
 |-- college_rush_att: integer (nullable = true)
 |-- college_rush_yds: integer (nullable = true)
 |-- college_rush_avg: float (nullable = true)
 |-- college_rush_TD: string (nullable = true)
 |-- college_rec_rec: string (nullable = true)
 |-- college_rec_yds: string (nullable = true)
 |-- college_rec_avg: string (nullable = true)
 |-- college_rec_TD: string (nullable = true)
 |-- college_scrim_plays: string (nullable = true)
 |-- college_scrim_yds: integer (nullable = true)
 |-- college_scrim_avg: string (nullable = true)
 |-- college_scrim_TD: integer (nullable = true)
 |-- Power_5_Num: string (nullable = true)



In [51]:
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.feature import VectorAssembler

# Load the saved model
model_path = "/content/drive/My Drive/Big Data/logistic_regression_model_drafted_rb"
loaded_model = LogisticRegressionModel.load(model_path)

# Assume combine_2024_rb is your new DataFrame
# Prepare features for the new DataFrame
assembler = VectorAssembler(inputCols=["forty", "college_rush_att", "college_rush_yds","college_rush_avg", "college_scrim_TD"],
                            outputCol="features")
new_data = assembler.transform(combine_2024_rb)

# Use the loaded model to make predictions on the new DataFrame
predictions = loaded_model.transform(new_data)

# Show the predictions
predictions.select("player", "prediction", "probability").show(truncate=False)


+----------------+----------+----------------------------------------+
|player          |prediction|probability                             |
+----------------+----------+----------------------------------------+
|Emani Bailey    |1.0       |[0.4066145378626602,0.5933854621373398] |
|Trey Benson     |1.0       |[0.15659251564568624,0.8434074843543138]|
|Blake Corum     |1.0       |[0.1340786393418934,0.8659213606581067] |
|Re'Mahn Davis   |1.0       |[0.2548769457399,0.7451230542601]       |
|Audric Estimé   |1.0       |[0.4467487239841065,0.5532512760158935] |
|Isaac Guerendo  |1.0       |[0.16939193726110463,0.8306080627388954]|
|George Holani   |1.0       |[0.23317888239819712,0.7668211176018029]|
|Bucky Irving    |1.0       |[0.254925676670875,0.745074323329125]   |
|Dillon Johnson  |0.0       |[0.60033538098549,0.39966461901451]     |
|Jawhar Jordan   |1.0       |[0.38932512069383585,0.6106748793061642]|
|Tyrone Tracy Jr.|1.0       |[0.3119390887840139,0.6880609112159861] |
|MarSh

In [52]:
from pyspark.sql.functions import col

# Select desired columns and order by probability ascending
ordered_predictions = predictions.select("player", "prediction", "probability") \
                                 .orderBy(col("probability").asc())

# Show the ordered predictions
ordered_predictions.show(truncate=False)


+----------------+----------+----------------------------------------+
|player          |prediction|probability                             |
+----------------+----------+----------------------------------------+
|Jaylen Wright   |1.0       |[0.12629462546423423,0.8737053745357658]|
|Blake Corum     |1.0       |[0.1340786393418934,0.8659213606581067] |
|Trey Benson     |1.0       |[0.15659251564568624,0.8434074843543138]|
|Isaac Guerendo  |1.0       |[0.16939193726110463,0.8306080627388954]|
|Kimani Vidal    |1.0       |[0.17339639238329774,0.8266036076167023]|
|Keilan Robinson |1.0       |[0.18726126540762492,0.8127387345923751]|
|George Holani   |1.0       |[0.23317888239819712,0.7668211176018029]|
|Re'Mahn Davis   |1.0       |[0.2548769457399,0.7451230542601]       |
|Bucky Irving    |1.0       |[0.254925676670875,0.745074323329125]   |
|MarShawn Lloyd  |1.0       |[0.27681576226508076,0.7231842377349192]|
|Tyrone Tracy Jr.|1.0       |[0.3119390887840139,0.6880609112159861] |
|Micha

In [53]:
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml.feature import VectorAssembler

# Load the saved linear regression model
model_path_carav = "/content/drive/My Drive/Big Data/linear_regression_model_carav_rb"
loaded_lr_model = LinearRegressionModel.load(model_path_carav)

# Assuming combine_2024_rb is your new DataFrame
# Define the features used in training the model
features = ['forty',"college_rush_att", "college_rush_avg", "college_scrim_TD", "college_scrim_yds"]

# Create a vector assembler to combine features into a single vector column
assembler = VectorAssembler(inputCols=features, outputCol="features")

# Transform the new DataFrame using the vector assembler
new_data_df = assembler.transform(combine_2024_rb)

# Make predictions on the new DataFrame
new_predictions = loaded_lr_model.transform(new_data_df)

# Select the player and the predicted carav columns
predicted_players_new = new_predictions.select('player', 'prediction')

# Show the players with the predicted carav values for the new DataFrame
predicted_players_new.show()


+----------------+------------------+
|          player|        prediction|
+----------------+------------------+
|    Emani Bailey|14.557418798632355|
|     Trey Benson| 18.75363483564702|
|     Blake Corum|22.651178433564468|
|   Re'Mahn Davis| 20.79427836972117|
|   Audric Estimé|13.699186000494308|
|  Isaac Guerendo| 16.79794599202738|
|   George Holani|22.779697968290165|
|    Bucky Irving|23.026807357213357|
|  Dillon Johnson| 15.94496502121362|
|   Jawhar Jordan|15.457483317318946|
|Tyrone Tracy Jr.|20.049832235886157|
|  MarShawn Lloyd|15.337823400302213|
|  Kendall Milton|10.869586769492741|
| Keilan Robinson| 15.47364632362256|
|   Cody Schrader|11.829477688618596|
|    Kimani Vidal|24.154419223354864|
|   Michael Wiley|19.340228471190287|
|   Jaylen Wright|19.751380736698167|
+----------------+------------------+



In [54]:
from pyspark.sql.functions import col

# Order the predictions by the predicted carav values in ascending order
ordered_predictions = new_predictions.orderBy(col('prediction').desc())

# Select the player and the predicted carav columns
ordered_predicted_players = ordered_predictions.select('player', 'prediction')

# Show the ordered players with the predicted carav values for the new DataFrame
ordered_predicted_players.show()


+----------------+------------------+
|          player|        prediction|
+----------------+------------------+
|    Kimani Vidal|24.154419223354864|
|    Bucky Irving|23.026807357213357|
|   George Holani|22.779697968290165|
|     Blake Corum|22.651178433564468|
|   Re'Mahn Davis| 20.79427836972117|
|Tyrone Tracy Jr.|20.049832235886157|
|   Jaylen Wright|19.751380736698167|
|   Michael Wiley|19.340228471190287|
|     Trey Benson| 18.75363483564702|
|  Isaac Guerendo| 16.79794599202738|
|  Dillon Johnson| 15.94496502121362|
| Keilan Robinson| 15.47364632362256|
|   Jawhar Jordan|15.457483317318946|
|  MarShawn Lloyd|15.337823400302213|
|    Emani Bailey|14.557418798632355|
|   Audric Estimé|13.699186000494308|
|   Cody Schrader|11.829477688618596|
|  Kendall Milton|10.869586769492741|
+----------------+------------------+

